# Project 2

## Setup

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, floor, window, concat_ws
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType

In [2]:
# Initialize SparkSession with the Kafka JAR
spark = SparkSession.builder \
    .appName("KafkaTaxiStream") \
    .getOrCreate()

print("✅ Spark Session created successfully!")

✅ Spark Session created successfully!


In [3]:
# Define Schema for Incoming Data
schema = StructType() \
    .add("medallion", StringType()) \
    .add("hack_license", StringType()) \
    .add("pickup_datetime", TimestampType()) \
    .add("dropoff_datetime", TimestampType()) \
    .add("trip_time_in_secs", DoubleType()) \
    .add("trip_distance", DoubleType()) \
    .add("pickup_longitude", DoubleType()) \
    .add("pickup_latitude", DoubleType()) \
    .add("dropoff_longitude", DoubleType()) \
    .add("dropoff_latitude", DoubleType()) \
    .add("payment_type", StringType()) \
    .add("fare_amount", DoubleType()) \
    .add("surcharge", DoubleType()) \
    .add("mta_tax", DoubleType()) \
    .add("tip_amount", DoubleType()) \
    .add("tolls_amount", DoubleType())

In [4]:
# Read Kafka stream
taxi_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "taxi-trips") \
    .option("startingOffsets", "earliest") \
    .load()

# Parse JSON
parsed_taxi_stream = taxi_stream.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*")

# Query 0: Data Cleansing and setup

In [5]:
# remove malformed and invalid data
cleaned_taxi_stream = parsed_taxi_stream \
    .filter("medallion IS NOT NULL AND hack_license IS NOT NULL") \
    .filter("pickup_longitude != 0.0 AND pickup_latitude != 0.0") \
    .filter("dropoff_longitude != 0.0 AND dropoff_latitude != 0.0")

In [6]:
query = cleaned_taxi_stream.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("taxi_trips_cleaned") \
    .start()

In [20]:
# View cleansed stream in notebook
spark.sql("SELECT * FROM taxi_trips_cleaned").show(truncate=False)

+--------------------------------+--------------------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+
|medallion                       |hack_license                    |pickup_datetime    |dropoff_datetime   |trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|
+--------------------------------+--------------------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+
|07290D3599E7A0D62097A346EFCC1FB5|E7750A37CAB07D0DFF0AF7E3573AC141|2013-01-01 00:00:00|2013-01-01 00:02:00|120.0            |0.44         |-73.956528      |40.716976      |-73

# Query 1: Frequent Routes

## Part 1

In [8]:
from pyspark.sql.functions import udf

In [9]:
reference_lat = 41.474937
reference_lon = -74.913585
total_cells = 300

# Cell sizes from http://www.debs2015.org/call-grand-challenge.html
cell_size_lat_deg = 0.004491556  # 500m south
cell_size_lon_deg = 0.005986     # 500m east

In [10]:
# Main function to calculate how far the cell location is from the origin
def get_cell_id(lat, lon):
    if lat is None or lon is None:
        return None
    try:
        dx = int((lon - reference_lon) / cell_size_lon_deg) + 1 # how many cells east
        dy = int((reference_lat - lat) / cell_size_lat_deg) + 1 # how many cells south
        if 1 <= dx <= total_cells and 1 <= dy <= total_cells: # validate
            return f"{dx}.{dy}"
        else:
            return None
    except:
        return None

# create spark udf
get_cell_udf = udf(get_cell_id, StringType())

In [11]:
# Take cleaned taxi stream data and convert pickup and dropoff locations into start and end cell IDs
stream_with_cells = cleaned_taxi_stream \
    .withColumn("start_cell_id", get_cell_udf("pickup_latitude", "pickup_longitude")) \
    .withColumn("end_cell_id", get_cell_udf("dropoff_latitude", "dropoff_longitude")) \
    .filter("start_cell_id IS NOT NULL AND end_cell_id IS NOT NULL")

In [12]:
# Count the number of rides for each route in the last 30 minutes 
frequent_routes = stream_with_cells \
    .withWatermark("dropoff_datetime", "30 minutes") \
    .groupBy(
        window(col("dropoff_datetime"), "30 minutes"),
        col("start_cell_id"),
        col("end_cell_id")
    ) \
    .count() \
    .select("start_cell_id", "end_cell_id", "count") \
    .orderBy(col("count").desc())

In [13]:
# store results
frequent_routes.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("top_routes") \
    .start()

In [21]:
spark.sql("SELECT * FROM top_routes LIMIT 10").show(truncate=False)

+-------------+-----------+-----+
|start_cell_id|end_cell_id|count|
+-------------+-----------+-----+
|155.161      |155.161    |13   |
|156.161      |156.161    |9    |
|158.159      |157.161    |9    |
|156.163      |155.162    |8    |
|156.158      |156.158    |8    |
|156.163      |157.162    |8    |
|158.159      |157.162    |8    |
|156.163      |156.163    |8    |
|155.162      |155.162    |7    |
|154.160      |152.164    |7    |
+-------------+-----------+-----+



## Part 2

In [23]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [27]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

reference_lat = 41.474937
reference_lon = -74.913585
total_cells = 300
cell_size_lat_deg = 0.004491556  # 500m south
cell_size_lon_deg = 0.005986     # 500m east

# Cell ID calculation function
def get_cell_id(lat, lon):
    if lat is None or lon is None:
        return None
    try:
        dx = int((lon - reference_lon) / cell_size_lon_deg) + 1  # how many cells east
        dy = int((reference_lat - lat) / cell_size_lat_deg) + 1  # how many cells south
        if 1 <= dx <= total_cells and 1 <= dy <= total_cells:  # validate
            return f"{dx}.{dy}"
        else:
            return None
    except:
        return None

# Create UDF
get_cell_udf = udf(get_cell_id, StringType())

# Function to calculate delay
def process_batch(df, epoch_id):
    # Current time for delay calculation
    output_time = time.time() * 1000  # Convert to milliseconds
    
    if df.isEmpty():
        return
    
    # Get the latest event that triggered the update
    latest_event = df.orderBy(col("processing_time").desc()).first()
    
    # Calculate delay
    delay = output_time - latest_event["processing_time"]
    
    # Get current top 10 routes
    top_routes = spark.sql("""
        SELECT start_cell_id, end_cell_id, count
        FROM frequent_routes
        ORDER BY count DESC
        LIMIT 10
    """).collect()
    
    # Prepare the result with the 10 most frequent routes
    result = {
        "pickup_datetime": latest_event["pickup_datetime"],
        "dropoff_datetime": latest_event["dropoff_datetime"],
        "delay": delay
    }
    
    # Add all 10 routes (or NULL if fewer than 10 available)
    for i in range(10):
        if i < len(top_routes):
            result[f"start_cell_id_{i+1}"] = top_routes[i]["start_cell_id"]
            result[f"end_cell_id_{i+1}"] = top_routes[i]["end_cell_id"]
        else:
            result[f"start_cell_id_{i+1}"] = None
            result[f"end_cell_id_{i+1}"] = None
    
    # Create a DataFrame with the result
    result_df = spark.createDataFrame([result])
    
    # Append to results table
    result_df.write.mode("append").saveAsTable("top_routes_results")

# Add processing time column for delay calculation - fixed syntax
cleaned_taxi_stream = cleaned_taxi_stream.withColumn(
    "processing_time", unix_timestamp() * 1000
)

# Process stream with cell IDs
stream_with_cells = cleaned_taxi_stream \
    .withColumn("start_cell_id", get_cell_udf("pickup_latitude", "pickup_longitude")) \
    .withColumn("end_cell_id", get_cell_udf("dropoff_latitude", "dropoff_longitude")) \
    .filter("start_cell_id IS NOT NULL AND end_cell_id IS NOT NULL")

# Calculate frequent routes - store in memory table
frequent_routes_query = stream_with_cells \
    .withWatermark("dropoff_datetime", "30 minutes") \
    .groupBy(
        window(col("dropoff_datetime"), "30 minutes"),
        col("start_cell_id"),
        col("end_cell_id")
    ) \
    .count() \
    .select(
        col("start_cell_id"),
        col("end_cell_id"),
        col("count")
    ) \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("frequent_routes_new") \
    .start()

# Create a trigger to monitor changes in the stream
# This will trigger the output whenever any new data comes in that could affect the top 10
change_monitor = stream_with_cells \
    .withWatermark("dropoff_datetime", "30 minutes") \
    .writeStream \
    .foreachBatch(process_batch) \
    .outputMode("update") \
    .trigger(processingTime="1 second") \
    .start()


In [28]:
# Query to view the results
spark.sql("""
    SELECT 
        pickup_datetime, dropoff_datetime,
        start_cell_id_1, end_cell_id_1,
        start_cell_id_2, end_cell_id_2,
        start_cell_id_3, end_cell_id_3,
        start_cell_id_4, end_cell_id_4,
        start_cell_id_5, end_cell_id_5,
        start_cell_id_6, end_cell_id_6,
        start_cell_id_7, end_cell_id_7,
        start_cell_id_8, end_cell_id_8,
        start_cell_id_9, end_cell_id_9,
        start_cell_id_10, end_cell_id_10,
        delay
    FROM top_routes_results
    ORDER BY dropoff_datetime DESC
    LIMIT 10
""").show(truncate=False)

+-------------------+-------------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+----------------+--------------+----------------+
|pickup_datetime    |dropoff_datetime   |start_cell_id_1|end_cell_id_1|start_cell_id_2|end_cell_id_2|start_cell_id_3|end_cell_id_3|start_cell_id_4|end_cell_id_4|start_cell_id_5|end_cell_id_5|start_cell_id_6|end_cell_id_6|start_cell_id_7|end_cell_id_7|start_cell_id_8|end_cell_id_8|start_cell_id_9|end_cell_id_9|start_cell_id_10|end_cell_id_10|delay           |
+-------------------+-------------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-------------+---------------+-----------

## Part 2

# Query 2: Profitable Areas

## Part 1

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.functions import when
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import window, max as max_

In [16]:
# divide query 1 degrees by 2
cell_size_lat_deg = 0.002245778   # 250m south
cell_size_lon_deg = 0.002993      # 250m east
total_cells = 600

get_cell_udf = udf(get_cell_id, StringType())

In [17]:
# Create profit stream steps by calculating start cell, profit, apply 15-min watermark and compute median profit.
profit_stream = cleaned_taxi_stream \
    .withColumn("start_cell_id", get_cell_udf("pickup_latitude", "pickup_longitude")) \
    .filter("start_cell_id IS NOT NULL") \
    .withColumn("profit", when((col("fare_amount") >= 0) & (col("tip_amount") >= 0), col("fare_amount") + col("tip_amount"))) \
    .withWatermark("dropoff_datetime", "15 minutes") \
    .groupBy(window("dropoff_datetime", "15 minutes"), col("start_cell_id")) \
    .agg(
        expr("percentile_approx(profit, 0.5)").alias("median_profit"),
        max_("pickup_datetime").alias("pickup_datetime"),
        max_("dropoff_datetime").alias("dropoff_datetime")
    ) \
    .selectExpr("start_cell_id as cell_id", "median_profit", "window.end as profit_window_end", "pickup_datetime", "dropoff_datetime")

profit_stream.writeStream \
    .format("memory") \
    .queryName("profit_stream") \
    .outputMode("complete") \
    .trigger(processingTime="30 seconds") \
    .start()

In [18]:
# Create empty_taxies_stream by saving pickup table 
cleaned_taxi_stream \
    .select("medallion", "pickup_datetime") \
    .filter("medallion IS NOT NULL") \
    .withWatermark("pickup_datetime", "30 minutes") \
    .writeStream \
    .format("memory") \
    .queryName("pickup_table") \
    .outputMode("append") \
    .start()

pickup_static = spark.read.table("pickup_table")

# get all drop off in last 30 monites
dropoffs = cleaned_taxi_stream \
    .withColumn("dropoff_cell_id", get_cell_udf("dropoff_latitude", "dropoff_longitude")) \
    .filter("dropoff_cell_id IS NOT NULL") \
    .select("medallion", "dropoff_datetime", "dropoff_cell_id") \
    .withWatermark("dropoff_datetime", "30 minutes")

# calculate empty taxies based on datetimes
empty_taxis = dropoffs.alias("d").join(
    pickup_static.alias("p"),
    (col("d.medallion") == col("p.medallion")) & (col("p.pickup_datetime") > col("d.dropoff_datetime")),
    how="left_anti"
)

# calculate empty taxes per cell
empty_taxis_per_cell = empty_taxis \
    .groupBy(window("dropoff_datetime", "30 minutes"), col("dropoff_cell_id")) \
    .agg(expr("approx_count_distinct(medallion)").alias("empty_taxis")) \
    .selectExpr("dropoff_cell_id as cell_id", "empty_taxis", "window.end as empty_window_end")

#read to memory
empty_taxis_per_cell.writeStream \
    .format("memory") \
    .queryName("empty_taxis_stream") \
    .outputMode("complete") \
    .trigger(processingTime="30 seconds") \
    .start()

In [19]:
# join together
def process_batch(_, __):
    spark.sql("""
        SELECT 
            p.pickup_datetime, p.dropoff_datetime, p.cell_id AS profitable_cell_id,
            e.empty_taxis AS empty_taxies_in_cell, 
            p.median_profit AS median_profit_in_cell,
        CASE 
            WHEN e.empty_taxis = 0 THEN NULL 
            ELSE p.median_profit / e.empty_taxis 
            END AS profitability_of_cell
        FROM profit_stream p
        JOIN empty_taxis_stream e ON p.cell_id = e.cell_id
        ORDER BY profitability_of_cell DESC
        LIMIT 10

    """).show(truncate=False, n=50)

cleaned_taxi_stream.selectExpr("CAST(NULL AS STRING) as test") \
    .writeStream \
    .foreachBatch(process_batch) \
    .outputMode("update") \
    .trigger(processingTime="30 seconds") \
    .start()


+---------------+----------------+------------------+--------------------+---------------------+---------------------+
|pickup_datetime|dropoff_datetime|profitable_cell_id|empty_taxies_in_cell|median_profit_in_cell|profitability_of_cell|
+---------------+----------------+------------------+--------------------+---------------------+---------------------+
+---------------+----------------+------------------+--------------------+---------------------+---------------------+

+---------------+----------------+------------------+--------------------+---------------------+---------------------+
|pickup_datetime|dropoff_datetime|profitable_cell_id|empty_taxies_in_cell|median_profit_in_cell|profitability_of_cell|
+---------------+----------------+------------------+--------------------+---------------------+---------------------+
+---------------+----------------+------------------+--------------------+---------------------+---------------------+

+---------------+----------------+------------